실습 목표  

0. 추천시스템의 개념과 목적을 이해한다.
1. Implicit 라이브러리를 활용하여 Matrix Factorization(이하 MF) 기반의 추천 모델을 만들어 본다.
2. 음악 감상 기록을 활용하여 비슷한 아티스트를 찾고 아티스트를 추천해 본다.
3. 추천 시스템에서 자주 사용되는 데이터 구조인 CSR Matrix을 익힌다
4. 유저의 행위 데이터 중 Explicit data와 Implicit data의 차이점을 익힌다.
5. 새로운 데이터셋으로 직접 추천 모델을 만들어 본다.
  


# 8-9. 프로젝트 - Movielens 영화 추천 실습
이전 스텝에서 배운 MF 모델 학습 방법을 토대로, 내가 좋아할 만한 영화 추천 시스템을 제작해 보겠습니다.  
이번에 활용할 데이터셋은 추천시스템의 MNIST라고 부를만한 Movielens 데이터입니다.  

+ 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
+ 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해볼 수 있습니다.
  별점을 시청횟수로 해석해서 생각하겠습니다.
+ 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.
MovieLens 1M Dataset 데이터셋의 경우는 크기가 작아서 아래와 같이 직접 다운 받고 압축해제 하겠습니다.

## 1) 데이터 준비와 전처리
Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.  
  
터미널에서 
```
1) wget으로 데이터 다운로드
$ wget http://files.grouplens.org/datasets/movielens/ml-1m.zip

2) 다운받은 데이터를 작업디렉토리로 옮김
$ mv ml-1m.zip ~/aiffel/recommendata_iu/data

3) 작업디렉토리로 이동
$ cd ~/aiffel/recommendata_iu/data

4) 압축 해제
$ unzip ml-1m.zip
```

In [1]:
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares
import os
import numpy as np
import pandas as pd
from pandas import DataFrame

In [2]:
# 다운 받은 데이터 읽기
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
print(len(ratings))
ratings.head()

1000209


,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


1000209개의 영화 별점 data

In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


3 & 4 & 5 점 짜리 영화가 836478개, 84% 정도있고 이것만 분석할 거다.

In [4]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count1'}, inplace=True)

In [5]:
ratings['count1']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count1, Length: 836478, dtype: int64

In [6]:
# column 이름 중 rating -> count1 확인하기
ratings.tail()

,user_id,movie_id,count1,timestamp
1000203,6040,1090,3,956715518
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648
1000208,6040,1097,4,956715569


In [7]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
print(len(movies))
movies.tail() # 여기까지가 전처리

3883


,movie_id,title,genre
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama
3882,3952,"Contender, The (2000)",Drama|Thriller


영화의 총 개수는 3883개 인데 이게 3, 4, 5점 영화에 포함되어있는지도 확인해야 할 듯

## 2) 분석해 봅시다.
+ ratings에 있는 유니크한 영화 개수
+ rating에 있는 유니크한 사용자 수
+ 가장 인기 있는 영화 30개(인기순)

In [8]:
# ratings movie_id 에 맞쳐서 movies 의 title과 genre 합치기 (merging)
# df_merge_how_outer = pd.merge(df_left, df_right,    ...: how='outer',    ...: on='KEY')
df_merge = pd.merge(ratings, movies, on='movie_id')
df_merge

,user_id,movie_id,count1,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...,...
836473,5851,3607,5,957756608,One Little Indian (1973),Comedy|Drama|Western
836474,5854,3026,4,958346883,Slaughterhouse (1987),Horror
836475,5854,690,3,957744257,"Promise, The (Versprechen, Das) (1994)",Romance
836476,5938,2909,4,957273353,"Five Wives, Three Secretaries and Me (1998)",Documentary


In [9]:
# 사용하는 컬럼만 남겨줍니다.
using_cols = ['user_id', 'movie_id', 'count1', 'title']
df_merge = df_merge[using_cols]
print(len(df_merge))

836478


In [10]:
df_merge

,user_id,movie_id,count1,title
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975)
1,2,1193,5,One Flew Over the Cuckoo's Nest (1975)
2,12,1193,4,One Flew Over the Cuckoo's Nest (1975)
3,15,1193,4,One Flew Over the Cuckoo's Nest (1975)
4,17,1193,5,One Flew Over the Cuckoo's Nest (1975)
...,...,...,...,...
836473,5851,3607,5,One Little Indian (1973)
836474,5854,3026,4,Slaughterhouse (1987)
836475,5854,690,3,"Promise, The (Versprechen, Das) (1994)"
836476,5938,2909,4,"Five Wives, Three Secretaries and Me (1998)"


### ratings에 있는 유니크한 영화 개수

In [11]:
# ratings에 있는 유니크한 영화 개수
df_merge['movie_id'].nunique()

3628

### rating에 있는 유니크한 사용자 수

In [12]:
# rating에 있는 유니크한 사용자 수  
df_merge['user_id'].nunique()

6039

### 가장 인기 있는 영화 30개(인기순)

In [13]:
# 가장 인기 있는 영화 30개(인기순)
movie_count = df_merge.groupby('title')['count1'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

## 3) 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.

Silence of the Lambs, The (1991)  
Jurassic Park (1993)  
Matrix, The (1999)  
Ghostbusters (1984)  
Bug's Life, A (1998)    
좋아하는 영화를 df_merge에 행 추가하기  

In [14]:
movie_id_i_like = [593, 480, 42571, 2716, 2355]

title_i_like = ['Silence of the Lambs, The (1991)',
               'Jurassic Park (1993)',
               'Matrix, The (1999)',
               'Ghostbusters (1984)',
               'Bug\'s Life, A (1998)']

my_list = pd.DataFrame({'user_id':['inyu']*5, 'movie_id':movie_id_i_like, 'count1':[5]*5, 'title':title_i_like})

if not df_merge.isin({'user_id':['inyu']})['user_id'].any():
    df_merge = df_merge.append(my_list)
    
df_merge.tail()

,user_id,movie_id,count1,title
0,inyu,593,5,"Silence of the Lambs, The (1991)"
1,inyu,480,5,Jurassic Park (1993)
2,inyu,42571,5,"Matrix, The (1999)"
3,inyu,2716,5,Ghostbusters (1984)
4,inyu,2355,5,"Bug's Life, A (1998)"


In [18]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = df_merge['user_id'].unique()
title_unique = df_merge['title'].unique()
movie_unique = df_merge['title'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}

In [17]:
title_unique

array(["One Flew Over the Cuckoo's Nest (1975)",
       'James and the Giant Peach (1996)', 'My Fair Lady (1964)', ...,
       'Promise, The (Versprechen, Das) (1994)',
       'Five Wives, Three Secretaries and Me (1998)',
       'Identification of a Woman (Identificazione di una donna) (1982)'],
      dtype=object)

### 모델에 활용하기 위한 전처리 
우리가 다루는 데이터에 indexing 하기. 추천시스템, 자연어처리에서 자주 하는 작업   
(참고) pandas.DataFrame.unique()은 indexing 작업에서 매우 유용  

In [19]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = df_merge['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(df_merge):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    df_merge['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_title_data = df_merge['title'].map(movie_to_idx.get).dropna()
if len(temp_title_data) == len(df_merge):
    print('title column indexing OK!!')
    df_merge['title'] = temp_title_data
else:
    print('title column indexing Fail!!')

df_merge # data의 user_id와 title 컬럼 내 값들이 모두 정수 인덱스 값으로 잘 변경 => 훈련을 위한 전처리 완료!

user_id column indexing OK!!
title column indexing OK!!


,user_id,movie_id,count1,title
0,0,1193,5,0
1,1,1193,5,0
2,2,1193,4,0
3,3,1193,4,0
4,4,1193,5,0
...,...,...,...,...
0,6039,593,5,121
1,6039,480,5,107
2,6039,42571,5,124
3,6039,2716,5,243


row가 836178 -> 836483으로 5개 추가 되었음 알 수 있다.

## 4) CSR matrix를 직접 만들어 봅시다.
```
csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])  
where data, row_ind and col_ind satisfy the relationship a[row_ind[k], col_ind[k]] = data[k]., M,N은 matrix의 shape
```

In [20]:
from scipy.sparse import csr_matrix

num_user = df_merge['user_id'].nunique()
num_movie = df_merge['title'].nunique()

In [21]:
csr_data = csr_matrix((df_merge['count1'], (df_merge.user_id, df_merge.title)), shape = (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

Matrix Factorization 모델을 implicit 패키지를 사용하여 학습해 봅시다.  
이 패키지에 구현된 als(AlternatingLeastSquares) 모델을 사용하겠습니다.   
Matrix Factorization에서 쪼개진 두 Feature Matrix를 한꺼번에 훈련하는 것은 잘 수렴하지 않기 때문에, 한쪽을 고정시키고 다른 쪽을 학습하는 방식을 번갈아 수행하는 AlternatingLeastSquares 방식이 효과적인 것으로 알려져 있습니다.  

In [22]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분. 학습 내용과는 무관.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

AlternatingLeastSquares 클래스의 init 파라미터를 살펴보겠습니다.
1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지
2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지
3. use_gpu : GPU를 사용할 것인지
4. iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지
  
1,4를 늘릴수록 학습데이터를 잘 학습하게 되지만 과적합의 우려가 있으니 좋은 값을 찾아야 합니다.

In [23]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, 
                                    regularization=0.01, 
                                    use_gpu=False, 
                                    iterations=15, dtype=np.float32)

In [24]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [25]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

##  6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [26]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = df_merge['user_id'].unique()
movie_unique = df_merge['title'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}
print(len(user_unique))
user_unique

6040


array([   0,    1,    2, ..., 6037, 6038, 6039])

### 나와 내가 선호한 Matrix의 내적

In [27]:
user, movie = user_to_idx[6039], movie_to_idx[124]  # 124:Matrix, The (1999)
user_vector, movie_vector = als_model.user_factors[user], als_model.item_factors[movie]

print('ok')

ok


In [28]:
user_vector

array([-1.5236168e-02,  5.8052145e-02, -1.5299320e-01, -6.7701447e-01,
        6.1518973e-01, -1.5307081e-01, -1.2542813e-01, -2.8274587e-01,
       -5.4459000e-01,  2.9757455e-01,  2.2660138e-02, -7.8756177e-01,
        3.4810323e-01,  3.5542354e-01, -2.3750801e-01,  2.5224990e-01,
        6.1987126e-01,  4.5089072e-01,  1.3342901e+00, -4.5710754e-01,
        1.2116484e-01,  9.2568052e-01,  6.2104195e-02,  8.3778703e-01,
       -6.7921184e-02, -8.5200316e-01, -6.1360228e-01, -9.2960936e-01,
        1.1614891e-01, -2.8196093e-01,  4.4165310e-01,  8.8177514e-01,
        1.3272965e-01, -8.2561141e-03, -3.2144198e-01, -2.1706349e-01,
        1.4133475e+00, -9.0122201e-02,  2.5290608e-01,  6.2376118e-01,
       -2.7666166e-01,  4.0630397e-01,  1.4021793e-02, -1.6297716e-01,
        6.9792873e-01, -5.1046282e-01,  4.9828760e-02, -4.6364492e-01,
        1.9202955e-01, -4.0349093e-01, -9.2663074e-01, -3.6148775e-01,
       -7.7428639e-02, -1.4923804e-01,  3.6461383e-01, -4.5636100e-01,
      

In [29]:
movie_vector

array([ 0.01763114,  0.01650981,  0.0027351 , -0.0081425 ,  0.00813866,
        0.03084476,  0.00468973, -0.00360374, -0.01716575,  0.00415077,
        0.01490416, -0.00674123,  0.03584789,  0.03803024, -0.00059114,
        0.01068634,  0.01248632, -0.01112321,  0.02741348, -0.02403677,
       -0.00813989,  0.0298785 ,  0.03889809,  0.04003103,  0.0076413 ,
       -0.01340166, -0.01905148, -0.00280073,  0.01042582,  0.01144323,
        0.01778906,  0.01603164, -0.00409327,  0.01407538, -0.00175621,
        0.0041545 ,  0.01758534,  0.0128152 ,  0.00986454,  0.01377301,
       -0.01473454,  0.00772766, -0.01563272,  0.01178577,  0.05600795,
       -0.01151151,  0.01026739,  0.0093149 ,  0.02305096, -0.02138852,
       -0.01477145, -0.0221703 ,  0.0018959 , -0.01927603,  0.03047724,
        0.00244005,  0.00817861,  0.00850403, -0.00738319, -0.00046213,
       -0.00826979,  0.0110811 , -0.00644955, -0.01210919, -0.01800833,
       -0.01746245, -0.00659101,  0.01358022,  0.00187661,  0.03

In [30]:
# 내적 
np.dot(user_vector, movie_vector)

0.5003752

### 나와 선택하지 않은 영화와의 내적

In [31]:
movie = movie_to_idx[0]
movie_vector = als_model.item_factors[movie]

print('ok')

ok


In [32]:
# 내적 
np.dot(user_vector, movie_vector)

0.15195316

선호했던 영화인 Matrix가 훨씬 높게 나왔다. 

## 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

### Matrix와 비슷한 영화 찾기

In [35]:
movie_id = title_to_idx['Matrix, The (1999)']
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(124, 1.0),
 (92, 0.76602876),
 (62, 0.67572033),
 (141, 0.5692349),
 (200, 0.5557176),
 (145, 0.5401492),
 (107, 0.5385402),
 (375, 0.5173924),
 (317, 0.48700166),
 (175, 0.47558168),
 (75, 0.44990253),
 (44, 0.40533444),
 (479, 0.39636078),
 (3457, 0.39209622),
 (117, 0.38591465)]

In [36]:
idx_to_title = {v:k for k,v in title_to_idx.items()}
[idx_to_title[i[0]] for i in similar_movie]

['Matrix, The (1999)',
 'Terminator 2: Judgment Day (1991)',
 'Total Recall (1990)',
 'Fugitive, The (1993)',
 'Terminator, The (1984)',
 'Fifth Element, The (1997)',
 'Jurassic Park (1993)',
 'Face/Off (1997)',
 'Twelve Monkeys (1995)',
 'Men in Black (1997)',
 'Hunt for Red October, The (1990)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Contact (1997)',
 'Second Jungle Book: Mowgli & Baloo, The (1997)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)']

## 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.
filter_already_liked_items 는 유저가 이미 평가한 아이템은 제외하는 Argument입니다.

In [38]:
user = user_to_idx[6039]
# recommend에서는 user*item CSR Matrix를 받습니다.
recommended_movie = als_model.recommend(user, csr_data, N=15, filter_already_liked_items=True)
recommended_movie

[(92, 0.5772772),
 (175, 0.4486088),
 (38, 0.43748623),
 (50, 0.43075442),
 (22, 0.36044097),
 (62, 0.3603616),
 (40, 0.35440832),
 (87, 0.34632096),
 (670, 0.34260386),
 (157, 0.32908222),
 (51, 0.31989655),
 (48, 0.31678632),
 (201, 0.29305565),
 (141, 0.2830807),
 (23, 0.2759973)]

In [40]:
# idx_to_movie = {v:k for k, v in movie_to_idx.items()}
[idx_to_title[i[0]] for i in recommended_movie]

['Terminator 2: Judgment Day (1991)',
 'Men in Black (1997)',
 'Sixth Sense, The (1999)',
 'Toy Story 2 (1999)',
 'Back to the Future (1985)',
 'Total Recall (1990)',
 'Toy Story (1995)',
 'Braveheart (1995)',
 'Galaxy Quest (1999)',
 'Shawshank Redemption, The (1994)',
 'Fargo (1996)',
 'Saving Private Ryan (1998)',
 'Who Framed Roger Rabbit? (1988)',
 'Fugitive, The (1993)',
 "Schindler's List (1993)"]

### 기여도 확인

In [41]:
Terminator_2 = title_to_idx['Terminator 2: Judgment Day (1991)']
explain = als_model.explain(user, csr_data, itemid=Terminator_2)

In [42]:
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('Matrix, The (1999)', 0.2513350074092975),
 ('Jurassic Park (1993)', 0.22282198327825603),
 ('Silence of the Lambs, The (1991)', 0.06417000994382652),
 ("Bug's Life, A (1998)", 0.03286785882784831),
 ('Ghostbusters (1984)', -0.0028485334033038763)]

# 루브릭

평가문항
1. CSR matrix가 정상적으로 만들어졌다.  
사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.  
-> 6040x3628 sparse matrix 크기로 만들었다.   
  
2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.  
사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.  
-> 내가 5점을 준 영화는 0.5 정도가 나왔고, 내가 선택하지 않은 영화 0.15로 멀리 나왔다.   
   
3. 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.  
MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.  
-> 실제로 영화 'Men in Black'의 다른 편은 재미있게 봤었는데 추천해 떠서 한 번 찾아봐야겠다고 생각했다. 
아무래도 기여도가 큰 영화 'Matrix'와 비슷한 장르여서인거 같다.

# 고찰

우선 original data에 있던 'genre'를 활용하지 못한 점이 아쉬웠다. 나중에는 genre까지 이용해서 추천 시스템을 만들 수 있으면 좋겠다.  
해커톤과 설날까지 껴서 촉박한 감도 있었지만, 결과가 생각보다 유의미한 결과여서 뿌듯했다.  

***
+ als(AlternatingLeastSquares) 모델 복습
+ indexing code 스스로 쳐보기